Выполнить скрейпинг данных в веб-сайта http://books.toscrape.com/ и извлечь информацию о всех книгах на сайте во всех категориях: название, цену, количество товара в наличии (In stock (19 available)) в формате integer, описание.
Затем сохранить эту информацию в JSON-файле

In [1]:
# Импорт необходимых библиотек
from bs4 import BeautifulSoup
import requests
import pandas as pd
import urllib.parse
from fake_useragent import UserAgent
import json


In [2]:
# Путь к странице с данным
url_baza = "http://books.toscrape.com/catalogue/"
url = "http://books.toscrape.com/catalogue/page-1.html"
ua = UserAgent()
headers = {'User-Agent': ua.chrome}
session = requests.session()


## Скрейпинг нескольких страниц

In [3]:
# Пустые списки, которые будут содержать соответствующие данные:
# категория, название, цена, количество товара в наличии, описание.
category = []
name = []
price = []
price_val = []
amt = []
specification = []
# Создание пустого словаря, который будет содержать все получаемые данные
output = {}


In [4]:
while True:
    page = session.get(url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    try:
        next_page_link = soup.find('li', ('class', 'next'))
    except:
        next_page_link = 0
    # next_page_link = soup.find('li', ('class', 'next')).find('a')
    result = soup.find_all(
        'li', ('class', 'col-xs-6 col-sm-4 col-md-3 col-lg-3'))

    # Извлечение списка относительных ссылок на товары
    url_2 = []
    for i in result:
        for link in i.find_all('div', ('class', 'image_container')):
            url_2.append(link.find('a').get('href'))

    # Объединение двух частей ссылки в абсолютный путь
    # и создание списка со ссылками на каждый товар, расположенный на странице
    url_joined = []

    for link in url_2:
        url_joined.append(urllib.parse.urljoin(url_baza, link))

    for i in url_joined:
        response = session.get(i, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

    # Парсинг категории товара
        try:
            category.append(
                soup.find('ul', ('class', 'breadcrumb')).find_all('a')[2].text)
        except:
            category.append('')

    # Парсинг названия товара. Обработка исключения: добавляем пустую строку.
        try:
            name.append(
                soup.find('div', ('class', 'col-sm-6 product_main')).find('h1').text)
        except:
            name.append('')

    # Парсинг цены товара.
        try:
            p = soup.find('div', ('class', 'col-sm-6 product_main')
                          ).find('p', ('class', 'price_color')).text
            p_val = p[0]
            p = round(float(p[1:]), 2)
            price.append(p)
            price_val.append(p_val)
        except:
            price.append('')
            price_val.append('')

    # Парсинг количества товара
        try:
            amt.append(int(soup.find(
                'p', ('class', 'instock availability')).text.split('(')[1].split()[0]))
        except:
            amt.append('')

    # Парсинг описания товара
        try:
            spec = soup.find_all('p')
            specification.append(spec[3].text)
        except:
            specification.append('')

    # Записываем данные в словарь
        output = {'Название': name, 'Категория': category, 'Цена': price,
                  'Валюта': price_val, 'Количество в наличии': amt, 'Описание': specification}

    # Прерывание цикла
    if not next_page_link:
        break

    # Переход к следующей странице
    print(f"Обрабатывается страница {next_page_link.find('a')['href']}")
    url = url_baza + next_page_link.find('a')['href']


Обрабатывается страница page-2.html
Обрабатывается страница page-3.html
Обрабатывается страница page-4.html
Обрабатывается страница page-5.html
Обрабатывается страница page-6.html
Обрабатывается страница page-7.html
Обрабатывается страница page-8.html
Обрабатывается страница page-9.html
Обрабатывается страница page-10.html
Обрабатывается страница page-11.html
Обрабатывается страница page-12.html
Обрабатывается страница page-13.html
Обрабатывается страница page-14.html
Обрабатывается страница page-15.html
Обрабатывается страница page-16.html
Обрабатывается страница page-17.html
Обрабатывается страница page-18.html
Обрабатывается страница page-19.html
Обрабатывается страница page-20.html
Обрабатывается страница page-21.html
Обрабатывается страница page-22.html
Обрабатывается страница page-23.html
Обрабатывается страница page-24.html
Обрабатывается страница page-25.html
Обрабатывается страница page-26.html
Обрабатывается страница page-27.html
Обрабатывается страница page-28.html
Обрабатыв

In [163]:
# Запись результатов в файл
with open('baza.json', 'w') as fp:
    json.dump(output, fp)

In [5]:
# Преобразование в DataFrame
df = pd.DataFrame(output)
df

,Название,Категория,Цена,Валюта,Количество в наличии,Описание
0,A Light in the Attic,Poetry,51.77,£,22,It's hard to imagine a world without A Light i...
1,Tipping the Velvet,Historical Fiction,53.74,£,20,"""Erotic and absorbing...Written with starling ..."
2,Soumission,Fiction,50.10,£,20,"Dans une France assez proche de la nôtre, un h..."
3,Sharp Objects,Mystery,47.82,£,20,"WICKED above her hipbone, GIRL across her hear..."
4,Sapiens: A Brief History of Humankind,History,54.23,£,20,From a renowned historian comes a groundbreaki...
...,...,...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,Classics,55.53,£,1,\n\n\n\n\n\n
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",Sequential Art,57.06,£,1,High school student Kei Nagai is struck dead i...
997,A Spy's Devotion (The Regency Spies of London #1),Historical Fiction,16.97,£,1,"In England’s Regency era, manners and elegance..."
998,1st to Die (Women's Murder Club #1),Mystery,53.98,£,1,"James Patterson, bestselling author of the Ale..."
